In [1]:
import pandas as pd

## pre-2002 waves

In [2]:
income_pre2002 = pd.read_csv('income_pre2002.csv')

In [3]:
income_list = [f"{year}" for year in range(1978, 1994)]
new_columns = ['code','birthyear'] + income_list + ['1995','1997','1999']
income_pre2002.columns = new_columns

In [4]:
# convert birthyear to 19xx
income_pre2002['birthyear'] += 1900

## 2002 - 2020 waves

In [5]:
def income_calc(year):
    file_name = f'income_{year}.csv'
    df = pd.read_csv(file_name)
    
    df[f'{year}'] = df.iloc[:, 1].apply(
        lambda x: x if x >= 0 else pd.NA)

    # If reported in range
    df[f'{year}'] = df.apply(
        lambda row: (row.iloc[2] + row.iloc[3]) / 2 
                    if row.iloc[2] > 0 and row.iloc[3] > 0 
                    else (row.iloc[2] 
                            if row.iloc[2] > 0 and row.iloc[3] < 0 
                            else (row.iloc[3] 
                                if row.iloc[2] < 0 and row.iloc[3] > 0 
                                 else row[f'{year}'])),
        axis=1
    )
    
    # if reported less than 15000, all take 7500 (avg 15000 and 0)
    df[f'{year}'] = df.apply(
        lambda x: 7500 if x.iloc[4] == 0 else x[f'{year}'], axis=1)
    
    # if more than 50k, take average of 50k and truncated result (100k since 1989 https://nlsinfo.org/content/cohorts/nlsy79/topical-guide/income/income)
    df[f'{year}'] = df.apply(
        lambda x: 75000 if x.iloc[5] == 1 else x[f'{year}'], axis=1)
           
    # if reported in 10k, only 2002 wave
    if year == 2001: 
        df[f'{year}'] = df.apply(
            lambda x: 10000* x if x.iloc[6] >= 0 else x[f'{year}'], axis=1
        )    
    return df

In [6]:
dfs = []

for year in [2001, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019]:
    # Get the DataFrame for the given year
    df_year = income_calc(year)
    df_year = df_year[[f'{year}']]
    dfs.append(df_year)

# Concatenate all DataFrames along the columns
result_df = pd.concat(dfs, axis=1)

combine pre-2002 and 2002-2020

In [7]:
income = pd.concat([income_pre2002,result_df],axis=1)

In [8]:
income.head()

,code,birthyear,1978,1979,1980,1981,1982,1983,1984,1985,...,2001,2003,2005,2007,2009,2011,2013,2015,2017,2019
0,1,1958,4620,-5,5000,-5,-5,-5,-5,-5,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2,1959,4000,5000,6000,10000,11000,11500,11000,14000,...,0,0,5500,5000,6000,19000,21000,23000,25000,30000
2,3,1961,-4,7000,-5,7000,0,0,1300,0,...,0,<NA>,0,30000,<NA>,35000,40000,29000,80000,90000
3,4,1962,-4,-5,0,1086,70,0,0,7000,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,73000,0,2000
4,5,1959,2200,2000,3400,2300,2200,10500,-5,43119,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


## CPI adjust (2010 = 100)

In [9]:
CPI = pd.read_csv('../US_CPI.csv')

In [10]:
CPI

,Country Name,1978,1979,1980,1981,1982,1983,1984,1985,1986,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,United States,29.915931,33.282811,37.792366,41.6981,44.254788,45.676445,47.640776,49.329949,50.266255,...,108.566932,108.695722,110.067009,112.411557,115.157303,117.244196,118.690502,124.266414,134.211206,139.735794


In [11]:
def inflation_adjuster(df, CPI):
    for year in df.columns:
        if year.isdigit() and year in CPI.columns:
            df[year] = df[year] / CPI[year].values[0] / 0.01
    return df

In [12]:
adjusted_df = inflation_adjuster(income, CPI)

In [13]:
adjusted_df.columns

Index(['code', 'birthyear', '1978', '1979', '1980', '1981', '1982', '1983',
       '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992',
       '1993', '1995', '1997', '1999', '2001', '2003', '2005', '2007', '2009',
       '2011', '2013', '2015', '2017', '2019'],
      dtype='object')

## Pivot, calculate income by age

In [14]:
long_income = pd.melt(income, id_vars=['code', 'birthyear'], value_vars=['1978', '1979', '1980', '1981', '1982', '1983',
       '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992',
       '1993', '1995', '1997', '1999', '2001', '2003', '2005', '2007', '2009',
       '2011', '2013', '2015', '2017', '2019'], var_name='year', value_name='income', col_level=None, ignore_index=True)

In [15]:
def age_pivot():
    long_income['income'] = pd.to_numeric(long_income['income'], errors='coerce')
    long_income['income'] = long_income['income'].where(long_income['income'] > 0, pd.NA)
    long_income['age'] =  pd.to_numeric(long_income['year'])-long_income['birthyear']
    wide_df = long_income.pivot_table(index=['code', 'birthyear'], columns=['age'], values=['income'])
    wide_df = wide_df.drop(columns=[('income', 14), ('income', 15), ('income', 16), ('income', 17)])
    return wide_df

In [16]:
wide_df = age_pivot()

In [17]:
wide_df.to_csv('income_by_age_CPIadjusted.csv')

In [18]:
non_nan_count = wide_df.count()

print(non_nan_count)

        age
income  18     5705
        19     6926
        20     8001
        21     9087
        22     9166
        23     9238
        24     9129
        25     9001
        26     8612
        27     8149
        28     8016
        29     7739
        30     6787
        31     6391
        32     5358
        33     4950
        34     4150
        35     4076
        36     3274
        37     3284
        38     3187
        39     3194
        40     3188
        41     3100
        42     3119
        43     3099
        44     3081
        45     3033
        46     3054
        47     2951
        48     2936
        49     2827
        50     2804
        51     2657
        52     2602
        53     2569
        54     2490
        55     2411
        56     2398
        57     1809
        58     1646
        59     1053
        60      915
        61      436
        62      404
dtype: int64


## Spouse income
very similar structure to self income, only included married partner

In [19]:
sincome_pre2002 = pd.read_csv('../income_spouse/income_pre2002.csv')

In [20]:
sincome_pre2002.columns

Index(['R0000100', 'spouse_birth_year', 'R0155500', 'R0312710', 'R0482910',
       'R0784301', 'R1026201', 'R1412901', 'R1780701', 'R2143801', 'R2352501',
       'R2724701', 'R2973601', 'R3281601', 'R3561201', 'R3899301', 'R4314401',
       'R4996001', 'R5650801', 'R6374901', 'R6917801'],
      dtype='object')

In [21]:
new_column = ['code','spouse_birthyear'] + income_list + ['1995','1997','1999']
sincome_pre2002.columns = new_column

In [22]:
def spouse_income_calc(year):
    file_name = f'../income_spouse/income_{year}.csv' # only difference to self function--different file path
    df = pd.read_csv(file_name)
    
    df[f'{year}'] = df.iloc[:, 1].apply(
        lambda x: x if x >= 0 else pd.NA)

    # If reported in range
    df[f'{year}'] = df.apply(
        lambda row: (row.iloc[2] + row.iloc[3]) / 2 
                    if row.iloc[2] > 0 and row.iloc[3] > 0 
                    else (row.iloc[2] 
                            if row.iloc[2] > 0 and row.iloc[3] < 0 
                            else (row.iloc[3] 
                                if row.iloc[2] < 0 and row.iloc[3] > 0 
                                 else row[f'{year}'])),
        axis=1
    )
    
    # if reported less than 15000, all take 7500 (avg 15000 and 0)
    df[f'{year}'] = df.apply(
        lambda x: 7500 if x.iloc[4] == 0 else x[f'{year}'], axis=1)
    
    # if more than 50k, take average of 50k and truncated result (100k since 1989 https://nlsinfo.org/content/cohorts/nlsy79/topical-guide/income/income)
    df[f'{year}'] = df.apply(
        lambda x: 75000 if x.iloc[5] == 1 else x[f'{year}'], axis=1)
           
    # if reported in 10k, only 2002 wave
    if year == 2001: 
        df[f'{year}'] = df.apply(
            lambda x: 10000* x if x.iloc[6] >= 0 else x[f'{year}'], axis=1
        )    
    return df

In [23]:
dfs2 = []

for year in [2001, 2003, 2005, 2007, 2009, 2011, 2013, 2015, 2017, 2019]:
    # Get the DataFrame for the given year
    df_year = spouse_income_calc(year)
    df_year = df_year[[f'{year}']]
    dfs2.append(df_year)

# Concatenate all DataFrames along the columns
result_df2 = pd.concat(dfs2, axis=1)

In [36]:
# convert birthyear to 19xx
sincome_pre2002['spouse_birthyear'] = sincome_pre2002['spouse_birthyear'].apply(
    lambda x: x + 1900 if x>=0 else pd.NA
)

In [38]:
spouse_income = pd.concat([sincome_pre2002,result_df2],axis=1)
spouse_income = spouse_income.apply(pd.to_numeric, errors='coerce')
spouse_income[spouse_income < 0] = pd.NA

In [49]:
adjusted_df = inflation_adjuster(spouse_income, CPI)

In [50]:
spouse_income

,code,spouse_birthyear,1978,1979,1980,1981,1982,1983,1984,1985,...,2001,2003,2005,2007,2009,2011,2013,2015,2017,2019
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1946.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61574.407002,65194.396375,0.000000,0.000000,0.0,0.000000,0.000000,22999.985225,0.000000,0.000000
2,3,1960.0,NaN,NaN,NaN,23981.908164,21692.567873,9195.111445,37782.759505,60814.982961,...,22166.786521,NaN,44662.530404,47325.085034,NaN,53316.870841,46801.646215,45999.970450,44479.412261,46910.637892
3,4,1957.0,NaN,NaN,37044.518148,23981.908164,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65319.958038,24018.882621,46910.637892
4,5,1959.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36488.989777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12681,12682,1955.0,NaN,NaN,0.000000,3597.286225,2937.535233,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12682,12683,1961.0,1504.215253,0.000000,0.000000,0.000000,NaN,1972.570336,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12683,12684,1961.0,NaN,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12684,12685,NaN,NaN,NaN,1058.414804,2877.828980,9038.569947,13135.873493,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
non_nan_count = spouse_income.count()

print(non_nan_count)

code                12686
spouse_birthyear     8572
1978                 1200
1979                 1622
1980                 2211
1981                 2712
1982                 3363
1983                 3768
1984                 3621
1985                 4027
1986                 4325
1987                 4710
1988                 4975
1989                 5101
1990                 4474
1991                 4558
1992                 4484
1993                 4980
1995                 4916
1997                 4734
1999                 4329
2001                 4433
2003                 4045
2005                 3972
2007                 3964
2009                 3809
2011                 3634
2013                 3416
2015                 3769
2017                 3678
2019                 3501
dtype: int64


In [52]:
long_income2 = pd.melt(spouse_income, id_vars=['code', 'spouse_birthyear'], value_vars=['1978', '1979', '1980', '1981', '1982', '1983',
       '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992',
       '1993', '1995', '1997', '1999', '2001', '2003', '2005', '2007', '2009',
       '2011', '2013', '2015', '2017', '2019'], var_name='year', value_name='income', col_level=None, ignore_index=True)

In [42]:
long_income2.head()

,code,spouse_birthyear,year,income
0,1,NaN,1978,NaN
1,2,1946.0,1978,NaN
2,3,1960.0,1978,NaN
3,4,1957.0,1978,NaN
4,5,1959.0,1978,NaN


In [46]:
def spouse_age_pivot():
    long_income2['income'] = pd.to_numeric(long_income2['income'], errors='coerce')
    long_income2['income'] = long_income2['income'].where(long_income2['income'] > 0, pd.NA)
    long_income2['age'] =  pd.to_numeric(long_income2['year'])-long_income2['spouse_birthyear']
    wide_df2 = long_income2.pivot_table(index=['code', 'spouse_birthyear'], columns=['age'], values=['income'])
    columns_to_drop = [col for col in wide_df2.columns if col[0] == 'income' and -4 <= col[1] <= 17]
    wide_df2 = wide_df2.drop(columns=columns_to_drop)
    return wide_df2

In [47]:
wide_df2 = spouse_age_pivot()

In [53]:
wide_df.to_csv('spouse_income_by_age_CPIadjusted.csv')